# MindSpore

## OOP and Functional paradigms in MindSpore

MindStore можно писать в ООП, функциональном и оба сразу стилях

In [1]:
import mindspore.nn as nn
from mindspore import value_and_grad
import mindspore.numpy as mnp
from mindspore import grad, Tensor, Parameter
import numpy as np
import mindspore.nn as nn
import mindspore.ops as ops
import mindspore as ms


### OOP

In [2]:
class TrainOneStepCell(nn.Cell): # классы всегда должны наследоваться от nn.Cell
    def __init__(self, network, optimizer):
        super().__init__()
        self.network = network
        self.optimizer = optimizer
        self.grad_fn = value_and_grad(self.network, None, self.optimizer.parameters)

    def construct(self, *inputs):
        loss, grads = self.grad_fn(*inputs)
        self.optimizer(grads)
        return loss

network = nn.Dense(5, 3)
loss_fn = nn.BCEWithLogitsLoss()
network_with_loss = nn.WithLossCell(network, loss_fn)
optimizer = nn.SGD(network.trainable_params(), 0.001)
trainer = TrainOneStepCell(network_with_loss, optimizer)
trainer.grad_fn

<function mindspore.ops.composite.base._Grad.__call__.<locals>.after_grad(*args, **kwargs)>

### Functional programming

In [3]:
import mindspore.numpy as mnp
from mindspore import grad, Tensor

grad_tanh = grad(mnp.tanh)
point = Tensor(np.array([1.0]))

grad_tanh(point), grad(grad_tanh)(point)

(Tensor(shape=[1], dtype=Float64, value= [ 4.19974342e-01]),
 Tensor(shape=[1], dtype=Float64, value= [-6.39700008e-01]))

## Автоматическое дифференцирование

**Автоматическое дифференцирование**- метод дифференцирования функций, при которым мы не получаем аналитическое выражение, но получаем точно значение градиента в нужной точки. Работает он при условии того, у всех функций (в том числе и вложенных) существует значение в выбранной точке. Существует 2 режима этого дифференцирования: **прямой** и **обратный**. Функция grad (она в MindStore вычисляет градиент) работает во втором режиме. (как я понял, можно переключиться и в прямой режим, ожнако это лучше делать для сетей, у которых меньше входов, чем выходов)

![reverse_mode](./pictures/reverse_mode_AD.png)

## Dynamic and Static Graph

### Static

In [3]:
ms.set_context(mode=ms.GRAPH_MODE, device_target="CPU") 
'''
это означает, что используется статических граф и
все вычисление происходят на CPU
'''

class Net(nn.Cell):
    def __init__(self):
        super(Net, self).__init__()
        self.mul = ops.Mul()

    def construct(self, x, y):
        return self.mul(x, y)

x = ms.Tensor(np.array([1.0, 2.0, 3.0]).astype(np.float32))
y = ms.Tensor(np.array([4.0, 5.0, 6.0]).astype(np.float32))

net = Net()
print(net(x, y))

[ 4. 10. 18.]


[ERROR] CORE(40257,7edc4e585240,python):2024-10-09-17:43:09.994.803 [mindspore/core/utils/file_utils.cc:253] GetRealPath] Get realpath failed, path[/tmp/ipykernel_40257/3879228493.py]


### Dynamic

In [70]:
ms.set_context(mode=ms.PYNATIVE_MODE, device_target="GPU")
x = ms.Tensor(np.ones([1, 2, 3, 4]).astype(np.float32))
y = ms.Tensor(np.ones([1, 2, 3, 4]).astype(np.float32))
output = ops.add(x, y)
print(output.asnumpy())

[[[[2. 2. 2. 2.]
   [2. 2. 2. 2.]
   [2. 2. 2. 2.]]

  [[2. 2. 2. 2.]
   [2. 2. 2. 2.]
   [2. 2. 2. 2.]]]]


Немного о том, как это работает

In this example, when the Python interface ops.add(x, y) is called, the Python interface call is called to the C++ layer of the framework via Pybind11, and converted to C++ call. Then the framework will select the corresponding hardware device according to the device_target set by the users, and execute the add operation on that hardware device.

From the above principle, we can see that in PyNative mode, Python script code will be executed according to Python syntax, and the execution process involves MindSpore's API, which will be accelerated by executing on different hardware according to user settings. Therefore, in PyNative mode, users can use Python syntax and debugging methods at will, for example, you can use common IDEs such as PyCharm and VS Code to debug code.

Комбинировние

In [ ]:
class AddMulMul(nn.Cell):
    def __init__(self):
        super(AddMulMul, self).__init__()
        self.param = ms.Parameter(ms.Tensor(0.5, ms.float32))

    @ms.jit
    def construct(self, x):
        x = x + x
        x = x * self.param
        x = x * x
        return x

class CellCallSingleCell(nn.Cell):
    def __init__(self):
        super(CellCallSingleCell, self).__init__()
        self.conv = nn.Conv2d(1, 2, kernel_size=2, stride=1, padding=0, weight_init="ones", pad_mode="valid")
        self.bn = nn.BatchNorm2d(2, momentum=0.99, eps=0.00001, gamma_init="ones")
        self.relu = nn.ReLU()
        self.add_mul_mul = AddMulMul()

    def construct(self, x):
        x = self.conv(x)
        x = self.bn(x)
        x = self.add_mul_mul(x)
        x = self.relu(x)
        return x

ms.set_context(mode=ms.PYNATIVE_MODE, device_target="GPU")
inputs = ms.Tensor(np.ones([1, 1, 2, 2]).astype(np.float32))
net = CellCallSingleCell()
out = net(inputs)
print(out)

Вообщем, графы можно комбинировать. Для динамичесих графов используется статическая компиляция. Для статичесих - JIT-компиляция. Там, где подписано JIT - статический вычислительный граф.

## Static Graph Dynamic Shape

Менее проще я пока не могу описать: https://www.mindspore.cn/docs/en/r2.3.1/design/dynamic_shape.html